In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import pandas as pd
import pylhe
from tqdm import tqdm
import itertools

In [2]:
import os 
root_dirs = '../root/MG5_aMC_v2_7_3/SS2l_test/Events/'

In [3]:
root_name = ['run_01/tag_1_delphes_events.root', 'run_02/tag_1_delphes_events.root',
             'run_03/tag_1_delphes_events.root', 'run_04/tag_1_delphes_events.root',
             'run_05/tag_1_delphes_events.root', 'run_06/tag_1_delphes_events.root',
             'run_07/tag_1_delphes_events.root', 'run_08/tag_1_delphes_events.root',
             'run_09/tag_1_delphes_events.root', 'run_10/tag_1_delphes_events.root',
             'run_11/tag_1_delphes_events.root', 'run_12/tag_1_delphes_events.root',
             'run_13/tag_1_delphes_events.root', 'run_14/tag_1_delphes_events.root',
             'run_15/tag_1_delphes_events.root', 'run_16/tag_1_delphes_events.root',
             'run_17/tag_1_delphes_events.root', 'run_18/tag_1_delphes_events.root',
             'run_19/tag_1_delphes_events.root', 'run_20/tag_1_delphes_events.root',
             'run_21/tag_1_delphes_events.root', 'run_22/tag_1_delphes_events.root',
             'run_23/tag_1_delphes_events.root', 'run_24/tag_1_delphes_events.root',]
f_tA = [uproot.open(root_dirs + name) for i, name in enumerate(root_name)]

In [4]:
f_tA[0]['Delphes;1'].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Event                | int32_t                  | AsDtype('>i4')
Event/Event.fUniq... | uint32_t[]               | AsJagged(AsDtype('>u4'))
Event/Event.fBits    | uint8_t[]                | AsJagged(AsDtype('uint8'))
Event/Event.Number   | int64_t[]                | AsJagged(AsDtype('>i8'))
Event/Event.ReadTime | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.ProcTime | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Proce... | int32_t[]                | AsJagged(AsDtype('>i4'))
Event/Event.MPI      | int32_t[]                | AsJagged(AsDtype('>i4'))
Event/Event.Weight   | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Cross... | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Cross... | float[]                  | AsJagged(AsDtype('>f4'))
Event/Event.Scale    

In [54]:
def f(deltaeta, deltaphi):
    return np.sqrt(deltaeta**2 + deltaphi**2)
def function(f_Att):
    # Define Tree
    tree_electron_Att = f_Att['Delphes;1/Electron']
    tree_muon_Att = f_Att['Delphes;1/Muon']
    tree_jet_Att = f_Att['Delphes;1/Jet']
    tree_Emiss_Att = f_Att['Delphes;1/MissingET']
    
    # Define Branches
    Att_electron = tree_electron_Att.arrays(['Electron.Charge',
                                            'Electron.PT',
                                            'Electron.Eta',
                                            'Electron.Phi'], library='ak')
    Att_muon = tree_muon_Att.arrays(['Muon.Charge',
                                    'Muon.PT',
                                    'Muon.Eta',
                                    'Muon.Phi'], library='ak')
    Att_jet = tree_jet_Att.arrays(['Jet.Mass',
                                   'Jet.BTag',
                                   'Jet.PT',
                                   'Jet.Eta', 
                                   'Jet.Phi'], library='ak')
    Att_Emiss = tree_Emiss_Att.arrays(['MissingET.MET',
                                       'MissingET.Phi'], library='ak')
    
    # Define Variables
    EventSize = len(Att_jet['Jet.Mass'])
    jet_sort_pt = ak.sort(Att_jet['Jet.PT'], axis=-1, ascending=False)
    jet_num = ak.num(Att_jet['Jet.PT'], axis=-1)
    lepton_sort_pt = ak.sort(ak.concatenate([Att_electron['Electron.PT'], Att_muon['Muon.PT']], axis=-1), axis=-1, ascending=False)
    lepton_charge = ak.Array.tolist(ak.concatenate([Att_electron['Electron.Charge'], Att_muon['Muon.Charge']], axis=-1))
    lepton_eta = ak.concatenate([Att_electron['Electron.Eta'], Att_muon['Muon.Eta']], axis=-1)
    lepton_phi = ak.concatenate([Att_electron['Electron.Phi'], Att_muon['Muon.Phi']], axis=-1)
    bjet_pt = Att_jet['Jet.PT']*Att_jet['Jet.BTag']
    bjet_eta = Att_jet['Jet.Eta']*Att_jet['Jet.BTag']
    bjet_phi = Att_jet['Jet.Phi']*Att_jet['Jet.BTag']
    two_lepton = ak.where(ak.num(lepton_sort_pt)>=2)
    
    # Two leptons with same charge + at least 3 jets with 2 of them identified as b-jets
    event_signal = []
    for i in tqdm(two_lepton[0]): #Awkward Array has a length of 1
        num_jet, b_pt, lep_charge = jet_num[i], bjet_pt[i], lepton_charge[i]
        num_bjet = len(b_pt[ak.where(b_pt)])
        if lep_charge.count(-1) >= 2 or lep_charge.count(1) >=2:
            if num_jet >= 3 and num_bjet >=2:
                event_signal.append(i)
        
    
    # Transverse Momentum of leading and subleading b-jet
    event_jet = []
    for i in tqdm(event_signal):
        b_pt = bjet_pt[i]
        b_pt = b_pt[ak.where(b_pt)]
        b_pt = ak.sort(b_pt, ascending=False)
        if len(jet_sort_pt[i]) >= 2:
            if jet_sort_pt[i][0] > 30 and jet_sort_pt[i][1] > 20:
                event_jet.append(i)
        else:
            continue
    
    # Transverse Momentum of leading and subleading lepton
    event_lep = []
    for i in tqdm(event_jet):
        if len(lepton_sort_pt[i]) >= 2:
            if lepton_sort_pt[i][0] > 25 and lepton_sort_pt[i][1] > 20:
                event_lep.append(i)
        else:
            continue
    
    # Emiss
    event_emiss = []
    for i in tqdm(event_lep):
        if Att_Emiss['MissingET.MET'][i] > 30:
            event_emiss.append(i)
        else:
            continue
    
    # Eta for leptons & b-jets
    event_eta = []
    for i in tqdm(event_emiss):
        lep_eta, b_eta = lepton_eta[i], bjet_eta[i]
        lep_where = ak.where(np.abs(lep_eta)<2.5, 0, lep_eta)
        bjet_where = ak.where(np.abs(b_eta)<2.5, 0, b_eta)
        if ak.count_nonzero(lep_where) == 0 and ak.count_nonzero(bjet_where) == 0:
            event_eta.append(i)
 
    # delta R between b-jets and leptons, leptons and leptons, b-jets and b-jets
    event_deltaR = []
    for i in tqdm(event_eta):
        lep_eta, lep_phi, b_eta, b_phi = lepton_eta[i], lepton_phi[i], bjet_eta[i], bjet_phi[i]
        b_eta, b_phi = b_eta[ak.where(b_eta)], b_phi[ak.where(b_phi)]
        if len(b_eta) >= 2:
            deltaeta_bl = np.array([i-j for i in b_eta for j in lep_eta])
            deltaphi_bl = np.abs([i-j for i in b_phi for j in lep_phi])
            deltaphi_bl = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_bl])
            
            m = b_eta - b_eta[:, np.newaxis]
            deltaeta_bb = m[np.triu_indices(len(m[0]), k = 1)]
            m = b_phi - b_phi[:, np.newaxis]
            deltaphi_bb = np.abs(m[np.triu_indices(len(m[0]), k = 1)])
            deltaphi_bb = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_bb])
            
            m = lep_eta - lep_eta[:, np.newaxis]
            deltaeta_ll = m[np.triu_indices(len(m[0]), k = 1)]
            m = lep_phi - lep_phi[:, np.newaxis]
            deltaphi_ll = np.abs(m[np.triu_indices(len(m[0]), k = 1)])
            deltaphi_ll = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_ll])
            
            if min(f(deltaeta_bl, deltaphi_bl)) > 0.4 and min(f(deltaeta_bb, deltaphi_bb)) > 0.4 and min(f(deltaeta_ll, deltaphi_ll)) > 0.4:
                event_deltaR.append(i)
        else:
            continue
    
    # HT of 2 leading leptons and three leading jets
    event_ht = []
    for i in tqdm(event_deltaR):
        lep_pt, jet_pt = lepton_sort_pt[i], jet_sort_pt[i]
        if len(jet_pt) >= 3:
            ht = np.sum([lep_pt[0], lep_pt[1], jet_pt[0], jet_pt[1], jet_pt[2]])
            if ht > 300:
                event_ht.append(i)
                
    [EventSize, len(event_jet), len(event_lep), len(event_emiss), len(event_eta), len(event_deltaR), len(event_ht)]
    
    name = ['Input Event Size',
            'leading jets > 30 GeV & subleading jets > 20 GeV',
            'leading leptons > 25 GeV & subleading leptons > 20 GeV',
            r'$E_{T}^{miss}$',
            r'$|η^{l}|, |η^{b}| < 2.5$',
            r'Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4',
            r'$H_{T}$ (2 leading leptons & 3 leading jets) > 300 GeV']
    number = [EventSize, len(event_jet), len(event_lep), len(event_emiss), len(event_eta), len(event_deltaR), len(event_ht)]
    fd = pd.DataFrame({
            'Selection Cut (SS2l)': name,
            'Number of Event': number
            })
    return fd



            
function(f_tA[1])

100%|██████████| 121/121 [00:00<00:00, 1724.19it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,1000
1,leading jets > 30 GeV & subleading jets > 20 GeV,169
2,leading leptons > 25 GeV & subleading leptons ...,143
3,$E_{T}^{miss}$,124
4,"$|η^{l}|, |η^{b}| < 2.5$",122
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",121
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,110


In [36]:
l1 = ak.Array([[1, 1, 1, -1], [2, 1, 1, -1, -1], [1, 2, 1, -1, -1, -1]])
ak.count(l1, axis=-1)

<Array [4, 5, 6] type='3 * int64'>

In [43]:
l1 = [1, 2, 1, 1, -1, -1]
l1.count(-1)

2

In [106]:
function(f_tA[4])

100%|██████████| 776/776 [00:00<00:00, 2125.65it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4999
2,leading leptons > 25 GeV & subleading leptons ...,1293
3,$E_{T}^{miss}$,1174
4,"$|η^{l}|, |η^{b}| < 2.5$",1164
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",776
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,759


In [107]:
function(f_tA[5])

100%|██████████| 765/765 [00:00<00:00, 2035.93it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4999
2,leading leptons > 25 GeV & subleading leptons ...,1278
3,$E_{T}^{miss}$,1176
4,"$|η^{l}|, |η^{b}| < 2.5$",1159
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",765
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,747


In [108]:
function(f_tA[6])

100%|██████████| 69/69 [00:00<00:00, 1940.23it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,500
1,leading jets > 30 GeV & subleading jets > 20 GeV,500
2,leading leptons > 25 GeV & subleading leptons ...,121
3,$E_{T}^{miss}$,113
4,"$|η^{l}|, |η^{b}| < 2.5$",112
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",69
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,66


In [109]:
function(f_tA[7])

100%|██████████| 756/756 [00:00<00:00, 2116.92it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,5000
2,leading leptons > 25 GeV & subleading leptons ...,1221
3,$E_{T}^{miss}$,1137
4,"$|η^{l}|, |η^{b}| < 2.5$",1118
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",756
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,745


In [110]:
function(f_tA[8])

100%|██████████| 531/531 [00:00<00:00, 2214.76it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4989
2,leading leptons > 25 GeV & subleading leptons ...,1098
3,$E_{T}^{miss}$,976
4,"$|η^{l}|, |η^{b}| < 2.5$",965
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",531
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,440


In [111]:
function(f_tA[9])

100%|██████████| 605/605 [00:00<00:00, 2135.86it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4998
2,leading leptons > 25 GeV & subleading leptons ...,1208
3,$E_{T}^{miss}$,1066
4,"$|η^{l}|, |η^{b}| < 2.5$",1056
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",605
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,546


In [113]:
function(f_tA[10])

100%|██████████| 657/657 [00:00<00:00, 1893.01it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4999
2,leading leptons > 25 GeV & subleading leptons ...,1218
3,$E_{T}^{miss}$,1088
4,"$|η^{l}|, |η^{b}| < 2.5$",1079
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",657
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,609


In [114]:
function(f_tA[11])

100%|██████████| 668/668 [00:00<00:00, 2095.61it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4999
2,leading leptons > 25 GeV & subleading leptons ...,1260
3,$E_{T}^{miss}$,1151
4,"$|η^{l}|, |η^{b}| < 2.5$",1140
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",668
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,649


In [115]:
function(f_tA[12])

100%|██████████| 684/684 [00:00<00:00, 2392.46it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,5000
2,leading leptons > 25 GeV & subleading leptons ...,1216
3,$E_{T}^{miss}$,1099
4,"$|η^{l}|, |η^{b}| < 2.5$",1089
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",684
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,661


In [116]:
function(f_tA[13])

100%|██████████| 718/718 [00:00<00:00, 2113.38it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,5000
2,leading leptons > 25 GeV & subleading leptons ...,1258
3,$E_{T}^{miss}$,1175
4,"$|η^{l}|, |η^{b}| < 2.5$",1165
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",718
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,703


In [117]:
function(f_tA[14])

100%|██████████| 715/715 [00:00<00:00, 2233.38it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4999
2,leading leptons > 25 GeV & subleading leptons ...,1232
3,$E_{T}^{miss}$,1135
4,"$|η^{l}|, |η^{b}| < 2.5$",1117
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",715
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,703


In [118]:
function(f_tA[15])

100%|██████████| 717/717 [00:00<00:00, 2268.13it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,5000
2,leading leptons > 25 GeV & subleading leptons ...,1240
3,$E_{T}^{miss}$,1143
4,"$|η^{l}|, |η^{b}| < 2.5$",1134
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",717
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,706


In [119]:
function(f_tA[16])

100%|██████████| 475/475 [00:00<00:00, 1410.91it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4995
2,leading leptons > 25 GeV & subleading leptons ...,1182
3,$E_{T}^{miss}$,1038
4,"$|η^{l}|, |η^{b}| < 2.5$",1027
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",475
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,405


In [120]:
function(f_tA[17])

100%|██████████| 518/518 [00:00<00:00, 2072.84it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4994
2,leading leptons > 25 GeV & subleading leptons ...,1155
3,$E_{T}^{miss}$,1032
4,"$|η^{l}|, |η^{b}| < 2.5$",1020
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",518
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,469


In [121]:
function(f_tA[18])

100%|██████████| 637/637 [00:00<00:00, 1842.32it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4994
2,leading leptons > 25 GeV & subleading leptons ...,1258
3,$E_{T}^{miss}$,1133
4,"$|η^{l}|, |η^{b}| < 2.5$",1120
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",637
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,603


In [122]:
function(f_tA[19])

100%|██████████| 581/581 [00:00<00:00, 2331.79it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4999
2,leading leptons > 25 GeV & subleading leptons ...,1192
3,$E_{T}^{miss}$,1086
4,"$|η^{l}|, |η^{b}| < 2.5$",1072
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",581
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,549


In [123]:
function(f_tA[20])

100%|██████████| 636/636 [00:00<00:00, 2406.94it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4998
2,leading leptons > 25 GeV & subleading leptons ...,1221
3,$E_{T}^{miss}$,1113
4,"$|η^{l}|, |η^{b}| < 2.5$",1103
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",636
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,617


In [124]:
function(f_tA[21])

100%|██████████| 636/636 [00:00<00:00, 2456.41it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,4997
2,leading leptons > 25 GeV & subleading leptons ...,1224
3,$E_{T}^{miss}$,1126
4,"$|η^{l}|, |η^{b}| < 2.5$",1110
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",636
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,621


In [130]:
function(f_tA[22])

100%|██████████| 631/631 [00:00<00:00, 2332.29it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,5000
1,leading jets > 30 GeV & subleading jets > 20 GeV,5000
2,leading leptons > 25 GeV & subleading leptons ...,1233
3,$E_{T}^{miss}$,1140
4,"$|η^{l}|, |η^{b}| < 2.5$",1127
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",631
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,617


In [131]:
function(f_tA[23])

100%|██████████| 86/86 [00:00<00:00, 1976.93it/s]


,Selection Cut (SS2l),Number of Event
0,Input Event Size,680
1,leading jets > 30 GeV & subleading jets > 20 GeV,680
2,leading leptons > 25 GeV & subleading leptons ...,165
3,$E_{T}^{miss}$,152
4,"$|η^{l}|, |η^{b}| < 2.5$",150
5,"Δ$R_{bb}, ΔR_{bl}, ΔR_{ll}$ > 0.4",86
6,$H_{T}$ (2 leading leptons & 3 leading jets) >...,85
